
"""
Performance Variables 

- Wins 
- Places
- Win as Favs 
- Place as Favs
- Wins as Underdogs
- Place as Underdogs

"""


# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:
DEBUG = True

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

#fname_race = 'race_details_2022Q1_2022Q4.csv'
#fname_dogs = 'dog_results_2022Q1_2022Q2.csv'

#race_details = pd.read_csv(f'{fileloc}/{fname_race}')
#dogs_details = pd.read_csv(f'{fileloc}/{fname_dogs}')

#print(race_details.shape)
#print(dogs_details.shape)


## IMPORTING THE RAW DATA AND MERGE

In [8]:

# Importing Dog Race Results

#res_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\dog_results_20211201_20211231.csv')
#df.res_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\dog_results_20220101_20220531.csv')

print(f'{fileloc}/dog_results_2022Q1_2022Q2.csv')

df.res_raw_2021Q1Q4 = pd.read_csv(f'{fileloc}/dog_results_2021Q1_2021Q4.csv')
print(df.res_raw_2021Q1Q4.columns.values.tolist())
print(df.res_raw_2021Q1Q4.shape)

df.res_raw_2022Q1Q2 = pd.read_csv(f'{fileloc}/dog_results_2022Q1_2022Q2.csv')
print(df.res_raw_2022Q1Q2.columns.values.tolist())
print(df.res_raw_2022Q1Q2.shape)



/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/dog_results_2022Q1_2022Q2.csv
['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(435715, 19)
['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(320052, 19)


In [9]:

df.res_raw = df.res_raw_2021Q1Q4.append( df.res_raw_2022Q1Q2 )
print(df.res_raw.columns.values.tolist())
print(df.res_raw.shape)


['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(755767, 19)


In [10]:

df.res_raw.loc[:,"DogName"] = df.res_raw["DogName"].str.replace("'","").str.replace(".","")
if DEBUG: print(df.res_raw.shape) 
# 320 052

#if DEBUG: res_raw[ ~(res_raw.Place.isin( ['D','F','N','R','S','T',''])) ].shape # 66608
if DEBUG: print(df.res_raw[ ~(df.res_raw.Place.isin( ['R','S'])) ].shape) 
# 275 864

df.res_raw2 = df.res_raw[ ~(df.res_raw.Place.isin( ['R','S']))  ]
df.res_ = df.res_raw2[~df.res_raw2.Place.isna()]
if DEBUG: print(df.res_.shape) 
# 273 735

del df.res_raw, df.res_raw2

#res_ = res_raw


(755767, 19)
(649399, 19)
(644761, 19)


In [11]:

# Importing Race Details 

#race_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\race_details_20211201_20211231.csv', parse_dates = True)
#df.race_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\race_details_20220101_20220531.csv', parse_dates = True)

print(f'{fileloc}/race_details_2022Q1_2022Q2.csv')

df.race_raw_2022Q1Q2 = pd.read_csv(f'{fileloc}/race_details_2022Q1_2022Q2.csv', parse_dates = True)
print(df.race_raw_2022Q1Q2.columns.values.tolist())
print(df.race_raw_2022Q1Q2.shape)

df.race_raw_2021Q1Q4 = pd.read_csv(f'{fileloc}/race_details_2021Q1_2021Q4.csv', parse_dates = True)
print(df.race_raw_2021Q1Q4.columns.values.tolist())
print(df.race_raw_2021Q1Q4.shape)


/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/race_details_2022Q1_2022Q2.csv
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(38562, 8)
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(51833, 8)


In [12]:


df.race_raw = df.race_raw_2021Q1Q4.append( df.race_raw_2022Q1Q2 )
print(df.race_raw.columns.values.tolist())
print(df.race_raw.shape)




df.race_raw['Event_Dt'] = pd.to_datetime(df.race_raw['date']).dt.date

#dtvalct = race_raw.Event_Dt.value_counts()

df.race_raw.loc[df.race_raw.Track.isin(['Murray Bridge (MBR)','Murray Bridge (MBS)']), "Track"] = 'Murray Bridge'
df.race_raw.loc[df.race_raw.Track.isin(['Richmond (RIS)']), "Track"] = 'Richmond'

df.race_raw.loc[df.race_raw.Track.isin(['Sandown (SAP)']), "Track"] = 'Sandown Park'

df.race_raw.loc[df.race_raw.Track.isin(['Meadows (MEP)']), "Track"] = 'Meadows'
df.race_raw.loc[df.race_raw.Track.isin(['The Meadows']), "Track"] = 'Meadows'


if DEBUG : print(df.race_raw.shape)

# 2021
# 52,103 Races

# 2022 upto May
# 21,002 Races

# 2022 Q1/Q2
# 38,562 Races

# 2022 Q1/Q2 + 2021 Q1/Q2/Q3/Q4
# 81,412 Races

df.race_ = df.race_raw[ ~df.race_raw.Track.str.contains('NZ')]

if DEBUG : print(df.race_.shape)

del df.race_raw

#if DEBUG: print(df.race_.dtypes)


['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(90395, 8)
(90395, 9)
(81412, 9)


In [13]:

if DEBUG:
    print(df.res_.columns.values.tolist())
    print(df.race_.columns.values.tolist())
    
    print(df.res_.shape)
    print(df.race_.shape)

df.ft_raw = pd.merge(df.res_, df.race_, left_on = 'RaceId', right_on = '@id', how = 'inner', suffixes=('_DOG', '_RACE'))
if DEBUG: print(df.ft_raw.shape)

# 2021
# 333,285

# 2022 upto May
# 132,558

# 2022 Q1/Q2
# 245,827

# 2022 Q1/Q2 + 2021 Q1/Q2/Q3/Q4
# 245,827

#qc_DogName = df.ft_merg.DogName.value_counts()


['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt']
(644761, 19)
(81412, 9)
(577317, 28)


In [14]:

#summ.ft = hd.describe_df(df.ft)


In [ ]:

print(df.ft_raw.columns.values.tolist())


### A) Prepare the Base Dataset -- Exclusions (Missing ft sp, place info, Margin1, num_dogs u5)

In [15]:

# Delete races where ft_sp is blank
all_races = df.ft_raw['@id_RACE'].unique().tolist()
print(len(all_races))
print()

print("# Part 1 - Removing Races with missing fast track prices - can't be measured for performance")
# Clean the fast track prices
df.ft_raw['ft_sp'] = df.ft_raw.StartPrice.apply(lambda x : float(str(x).replace("$","").replace("F","")) )

blank_ft_sp_races = df.ft_raw[df.ft_raw['ft_sp'].isna()]['@id_RACE'].unique().tolist()
#print(len(blank_ft_sp_races))
df_base0 = df.ft_raw[ ( ~df.ft_raw['@id_RACE'].isin(blank_ft_sp_races) ) ]
print(f"Shape : {df_base0.shape}, Races : {len(df_base0['@id_RACE'].unique().tolist())}, Race w missing fast track prices : {len(blank_ft_sp_races)}")
print()

print("# Part 2 - Removing Races with Missing Place Information")
place_to_remove = ['F','T','N','P','B','D']
races_excl_pos = df_base0[df_base0['Place'].isin(place_to_remove)]['@id_RACE'].unique().tolist()
#print(len(races_excl_pos))

df_base1 = df_base0[ ( ~df_base0['@id_RACE'].isin(races_excl_pos) ) ]
print(f"Shape : {df_base1.shape}, Races : {len(df_base1['@id_RACE'].unique().tolist())}, Races w missing Place : {len(races_excl_pos)}")
print()

print("# Part 3 - Removing Races with no Missing Margin1 information")
races_excl_margin = df_base1[df_base1['Margin1'].isna()]['@id_RACE'].unique().tolist()
#print(len(races_excl_margin))

df_base2 = df_base1[ ( ~df_base1['@id_RACE'].isin(races_excl_margin) ) ]
print(f"Shape : {df_base2.shape}, Races : {len(df_base2['@id_RACE'].unique().tolist())}, Races w missing Margin : {len(races_excl_margin)}")

del df_base0, df_base1


80909

# Part 1 - Removing Races with missing fast track prices - can't be measured for performance
Shape : (569648, 29), Races : 79518, Race w missing fast track prices : 1391

# Part 2 - Removing Races with Missing Place Information
Shape : (541552, 29), Races : 75740, Races w missing Place : 3778

# Part 3 - Removing Races with no Missing Margin1 information
Shape : (541527, 29), Races : 75721, Races w missing Margin : 19


In [16]:

print("# Part 4 - Removing Races with under 5 dogs")
df_num_dogs = df_base2.groupby("@id_RACE").agg( { '@id_DOG':'nunique' } ).reset_index().rename(columns = {'@id_DOG':'x_num_dogs'})

print(df_num_dogs.shape)
print(df_base2.shape)
print()

df_base3 = pd.merge( df_base2, df_num_dogs, on = ["@id_RACE"], how = 'left')
print(df_base3.columns.values.tolist())
print()

races_u5dogs = df_base3[df_base3.x_num_dogs.isin([1,2,3,4])]['@id_RACE'].unique().tolist()
print(len(races_u5dogs))
print()

print( len(df_base3['@id_RACE'].unique().tolist()))
df_base = df_base3[ ( ~df_base3['@id_RACE'].isin(races_u5dogs) ) ].copy()   
print()

#print(df.ft.shape)
print(f"Shape : {df_base.shape}, Races : {len(df_base['@id_RACE'].unique().tolist())}, Races u5 Dogs : {len(races_u5dogs)}")
print()

del df_base2, df_base3


# Part 4 - Removing Races with under 5 dogs
(75721, 2)
(541527, 29)

['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs']

1243

75721

Shape : (536782, 30), Races : 74478, Races u5 Dogs : 1243



In [17]:

#del dognameslist
# DogNames list
dognameslist = df_base.DogName.unique().tolist()

# Fix the Place Variable
df_base['position'] = df_base.Place.apply(lambda x : x.replace("=","") if x in ['1=','2=','3=','4=','5=','6=','7=','8='] else x)

#print(df_base_sp.position.value_counts())


### B) Create Variable for Performance

In [18]:

df_base_perf = df_base.copy()


In [20]:

# Sort for Easy QC
df_base_perf.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)


In [21]:

#del dognameslist

# DogNames list
dognameslist = df_base_perf.DogName.unique().tolist()

# Fix the Place Variable
df_base_perf['position'] = df_base_perf.Place.apply(lambda x : x.replace("=","") if x in ['1=','2=','3=','4=','5=','6=','7=','8='] else x)

#print(df.ft.position.value_counts())


In [22]:

print(dognameslist[5])


BIG TOMMY NITRO


In [23]:

df_base_perf[ df_base_perf['@id_DOG'].isin([108390981])]


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position
3729,108390981,4,DOLLY DOES IT,5.0,5,30.4,$11.00,NaN,6.00,0.14,0,0,B3,NaN,26.43,80.0,626271182,92892,M Elphinstone,626271182,10,GREAT NORTHERN,03:14PM,452m,Mixed 4/5,Devonport,05 Jan 21,2021-01-05,11.0,8,4
9211,108390981,5,DOLLY DOES IT,4.0,4,30.2,$7.00,NaN,6.00,1.14,0,0,NaN,NaN,26.40,0.0,629797242,92892,M Elphinstone,629797242,10,GREAT NORTHERN,03:09PM,452m,Grade 5,Devonport,12 Jan 21,2021-01-12,7.0,7,5
14917,108390981,1,DOLLY DOES IT,5.0,5,30.2,$7.00,NaN,5.25,NaN,0,0,NaN,8.1,26.20,1890.0,629797513,92892,M Elphinstone,629797513,4,SUREPICK,01:27PM,452m,Grade 5,Devonport,19 Jan 21,2021-01-19,7.0,7,1
37110,108390981,7,DOLLY DOES IT,1.0,1,30.0,$7.00,NaN,10.50,2.00,0,0,NaN,NaN,26.48,0.0,638810808,92892,M Elphinstone,638810808,8,LADBROKES,02:22PM,452m,Mixed 4/5 Heat,Devonport,16 Feb 21,2021-02-16,7.0,8,7
48421,108390981,8,DOLLY DOES IT,8.0,8,29.9,$91.00,NaN,18.50,3.57,0,0,SO,NaN,26.91,0.0,643554706,92892,M Elphinstone,643554706,8,NEW RECRUIT 2012,02:22PM,452m,Grade 4,Devonport,02 Mar 21,2021-03-02,91.0,8,8


In [24]:

# ADDITIONAL VARIABLES
doglevel = ['@id_DOG']


In [25]:

# Total Races
df_base_perf.loc[:,"xr_races"] = df_base_perf.groupby(doglevel).cumcount()

#Wins by a Dog
df_base_perf.loc[:,"wins"] = df_base_perf['position'].apply(lambda x : 1 if x == '1' else 0 )

# Rolling wins for each Dog
df_base_perf.loc[:,"wins_prev"] = df_base_perf.groupby(doglevel)['wins'].shift(1)
df_base_perf.loc[:,"xr_wins"] = df_base_perf.groupby(doglevel)['wins_prev'].rolling(window=999, min_periods = 1).sum().reset_index(0,drop=True)

df_base_perf['xr_wins'].fillna(0, inplace = True)
df_base_perf.loc[:,"xr_wins_pct"] = df_base_perf['xr_wins'] / df_base_perf['xr_races']
df_base_perf['xr_wins_pct'].fillna(0, inplace = True)


In [26]:

#Top 2 Finish
df_base_perf.loc[:,"fin_t2"] = df_base_perf['position'].apply(lambda x : 1 if x in ['1','2'] else 0 )

# Rolling wins for each Dog
df_base_perf.loc[:,"fin_t2_prev"] = df_base_perf.groupby(doglevel)['fin_t2'].shift(1)
df_base_perf.loc[:,"xr_fin_t2"] = df_base_perf.groupby(doglevel)['fin_t2_prev'].rolling(window=999, min_periods = 1).sum().reset_index(0,drop=True)

df_base_perf['xr_fin_t2'].fillna(0, inplace = True)
df_base_perf.loc[:,"xr_fin_t2_pct"] = df_base_perf['xr_fin_t2'] / df_base_perf['xr_races']
df_base_perf['xr_fin_t2_pct'].fillna(0, inplace = True)

print(datetime.now())


2022-11-27 10:38:28.408161


In [27]:

#Create a Placed variable 
df_base_perf.loc[:,"fin_plc"] = df_base_perf.apply(lambda x : 1 if (x.position in ['1','2','3']) & (x.x_num_dogs >=8)\
                                    else 1 if (x.position in ['1','2']) & (x.x_num_dogs >= 5)\
                                    else 0 if (x.x_num_dogs < 5)\
                                    else 0, axis = 1)

# Rolling wins for each Dog
df_base_perf.loc[:,"fin_plc_prev"] = df_base_perf.groupby(doglevel)['fin_plc'].shift(1)
df_base_perf.loc[:,"xr_fin_plc"] = df_base_perf.groupby(doglevel)['fin_plc_prev'].rolling(window=999, min_periods = 1).sum().reset_index(0,drop=True)

df_base_perf['xr_fin_plc'].fillna(0, inplace = True)
df_base_perf.loc[:,"xr_fin_plc_pct"] = df_base_perf['xr_fin_plc'] / df_base_perf['xr_races']
df_base_perf['xr_fin_plc_pct'].fillna(0, inplace = True)

print(datetime.now())


2022-11-27 10:39:17.104357


In [28]:

df_base_perf[25:35]


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position,xr_races,wins,wins_prev,xr_wins,xr_wins_pct,fin_t2,fin_t2_prev,xr_fin_t2,xr_fin_t2_pct,fin_plc,fin_plc_prev,xr_fin_plc,xr_fin_plc_pct
25714,109032188,4,DESTINI TRUCKER,6.0,6,33.9,$11.00,NaN,5.37,1.97,M/43,4,NaN,8.56,23.01,NaN,637193352,130173,R Winter,637193352,12,THE OAKS FUNCTION CENTRE HT3,10:29PM,400m,Grade 5 Heat,Warragul,02 Feb 21,2021-02-02,11.0,7,4,3,0,0.0,0.0,0.000000,0,0.0,1.0,0.333333,0,1.0,2.0,0.666667
35301,109032188,6,DESTINI TRUCKER,8.0,9,33.8,$45.00,NaN,10.37,1.04,M/355,0,NaN,5.32,25.50,NaN,640627712,130173,R Winter,640627712,7,GRV VIC BRED SERIES HT2,07:52PM,440m,Grade 5 Heat,Sale,14 Feb 21,2021-02-14,45.0,8,6,4,0,0.0,0.0,0.000000,0,0.0,1.0,0.250000,0,0.0,2.0,0.500000
41053,109032188,1,DESTINI TRUCKER,1.0,1,34.1,$16.00,NaN,1.91,NaN,Q/111,0,NaN,5.27,25.15,NaN,643136707,130173,R Winter,643136707,10,MAFFRA POLY & PUMPS,08:47PM,440m,Grade 5,Sale,21 Feb 21,2021-02-21,16.0,8,1,5,1,0.0,0.0,0.000000,1,0.0,1.0,0.200000,1,0.0,2.0,0.400000
46717,109032188,4,DESTINI TRUCKER,2.0,2,34.4,$9.40,NaN,3.91,1.41,M/64,0,NaN,9.75,17.79,NaN,645386699,130173,R Winter,645386699,10,TAB - LONG MAY WE PLAY,04:52PM,311m,Grade 4,Cranbourne,28 Feb 21,2021-02-28,9.4,7,4,6,0,1.0,1.0,0.166667,0,1.0,2.0,0.333333,0,1.0,3.0,0.500000
161099,109032188,2,DESTINI TRUCKER,1.0,1,34.0,$7.30,NaN,0.43,0.43,M/11,1,NaN,9.89,18.13,NaN,691658567,130173,R Winter,691658567,12,DOWNLOAD THE WATCHDOG APP,10:34PM,311m,Grade 5,Cranbourne,14 Jul 21,2021-07-14,7.3,7,2,7,0,0.0,1.0,0.142857,1,0.0,2.0,0.285714,1,0.0,3.0,0.428571
167326,109032188,2,DESTINI TRUCKER,5.0,5,34.2,$8.00,NaN,0.43,0.43,M/22,4,NaN,9.82,17.95,NaN,694218186,130173,R Winter,694218186,12,Cranbournegreyhounds.com.au,10:27PM,311m,Grade 5,Cranbourne,21 Jul 21,2021-07-21,8.0,8,2,8,0,0.0,1.0,0.125000,1,1.0,3.0,0.375000,1,1.0,4.0,0.500000
173415,109032188,2,DESTINI TRUCKER,3.0,3,33.9,$5.70,NaN,0.10,0.10,Q/11,1,NaN,9.88,18.24,NaN,696879981,130173,R Winter,696879981,9,BACKMANS PET FOODS,09:27PM,311m,Grade 5,Cranbourne,28 Jul 21,2021-07-28,5.7,8,2,9,0,0.0,1.0,0.111111,1,1.0,4.0,0.444444,1,1.0,5.0,0.555556
179696,109032188,6,DESTINI TRUCKER,4.0,4,34.0,$8.80,NaN,7.33,2.04,M/87,4,NaN,10.38,18.60,NaN,699451616,130173,R Winter,699451616,12,WINNING POST SUPPLIES,10:27PM,311m,Grade 5,Cranbourne,04 Aug 21,2021-08-04,8.8,8,6,10,0,0.0,1.0,0.100000,0,1.0,5.0,0.500000,0,1.0,6.0,0.600000
187827,109032188,6,DESTINI TRUCKER,7.0,7,34.0,$2.70F,NaN,7.49,1.99,M/66,1,NaN,10.27,18.52,NaN,702793288,130173,R Winter,702793288,11,TAB.COM.AU,06:35PM,311m,Grade 5,Cranbourne,13 Aug 21,2021-08-13,2.7,7,6,11,0,0.0,1.0,0.090909,0,0.0,5.0,0.454545,0,0.0,6.0,0.545455
210493,109032188,4,DESTINI TRUCKER,1.0,1,34.0,$8.70,NaN,6.29,4.76,M/54,0,NaN,10.27,18.47,NaN,708963607,130173,R Winter,708963607,11,TAB VENUE MODE,06:07PM,311m,Grade 5,Cranbourne,08 Sep 21,2021-09-08,8.7,8,4,12,0,0.0,1.0,0.083333,0,0.0,5.0,0.416667,0,0.0,6.0,0.500000


In [29]:

#For the Wins in RaceType for the future

"""
# ADDITIONAL VARIABLES
grouplevel = ['@id_DOG','x_dist_gp']

# GROUPED X_DIST 
df.ft['x_dist_gp'] = df.ft['x_dist'].apply(lambda x : 300 if x < 330 else 350 if x < 380 else 400 if x < 420 else 450 if x < 485 else 500 if x < 550 else 600 if x < 650 else 700 if x < 750 else -999)
df.ft['x_runtm_gp'] = ( df.ft['RunTime'] / df.ft['x_dist'] ) * df.ft['x_dist_gp']

df.ft['x_rtgp_prev'] = df.ft.groupby(grouplevel)['x_runtm_gp'].shift(1)

#df.ft['x_rtgp_avg'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().reset_index(drop=True)
df.ft['x_rtgp_avg'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().values

#df.ft['x_rtgp_min'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().reset_index(drop=True)
df.ft['x_rtgp_min'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().values

#df.ft['x_rtgp_max'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)
df.ft['x_rtgp_max'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().values

df.ft['x_rtgp_range'] = df.ft['x_rtgp_max'] - df.ft['x_rtgp_min']

#df.ft['x_rtgp_std'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)
df.ft['x_rtgp_std'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().values

"""

print(datetime.now())


2022-11-27 10:39:17.148958


In [30]:

#df.ft.head(25)
#df.ft[ df.ft['@id_DOG'].isin([108390981,108391383,108391431]) ].head(25)
#df.ft[ df.ft['x_runtm_gp'].isna()].head()


In [31]:

print(df_base_perf.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'xr_races', 'wins', 'wins_prev', 'xr_wins', 'xr_wins_pct', 'fin_t2', 'fin_t2_prev', 'xr_fin_t2', 'xr_fin_t2_pct', 'fin_plc', 'fin_plc_prev', 'xr_fin_plc', 'xr_fin_plc_pct']


In [32]:

#temp = df.ft.groupby('x_dist').agg( {'@id_RACE':'nunique'} ).reset_index()
#temp.to_csv(f'{fileloc}/race_distance.csv', index = False)
#temp.sort_values(by = 'index').head()


In [33]:

print(pd.DataFrame(df_base_perf.dtypes).transpose())


  @id_DOG   Place DogName      Box    Rug   Weight StartPrice Handicap  Margin1  Margin2     PIR Checks Comments SplitMargin  RunTime Prizemoney RaceId TrainerId TrainerName @id_RACE RaceNum  \
0   int64  object  object  float64  int64  float64     object  float64  float64  float64  object  int64   object     float64  float64    float64  int64     int64      object    int64   int64   

  RaceName RaceTime Distance RaceGrade   Track    date Event_Dt    ft_sp x_num_dogs position xr_races   wins wins_prev  xr_wins xr_wins_pct fin_t2 fin_t2_prev xr_fin_t2 xr_fin_t2_pct fin_plc  \
0   object   object   object    object  object  object   object  float64      int64   object    int64  int64   float64  float64     float64  int64     float64   float64       float64   int64   

  fin_plc_prev xr_fin_plc xr_fin_plc_pct  
0      float64    float64        float64  


In [34]:

print(pd.DataFrame(df_base_perf.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  RunTime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    536782        0    74620  71107       0    259643        90362        0      167442       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  Track  date  Event_Dt  ft_sp  x_num_dogs  position  xr_races  wins  wins_prev  xr_wins  xr_wins_pct  fin_t2  fin_t2_prev  xr_fin_t2  xr_fin_t2_pct  \
0        15         0         0          0      0     0         0      0           0         0         0     0      25295        0            0       0        25295          0              0   

   fin_plc  fin_plc_prev  xr_fin_plc  xr_fin_plc_pct  
0        0         25295           0               0  



### Treat Missing Values


In [ ]:

"""
# All Columns Realted to Average Speed

df.ft['x_no_prev_races'] = df.ft['speed_prev'].apply( lambda x : 1 if pd.isna(x) else 0)

print( df.ft['x_no_prev_races'].value_counts() )

df.ft['x_speed_prev'] = df.ft['speed_prev'].fillna(0)
df.ft['x_speed_avg'] = df.ft['speed_avg'].fillna(0)
df.ft['x_speed_min'] = df.ft['speed_min'].fillna(0)
df.ft['x_speed_max'] = df.ft['speed_max'].fillna(0)

df.ft['x_speed_range'] = df.ft['speed_range'].fillna(-1)

df.ft['x_speed_std'] = df.ft['speed_std'].fillna(0)
df.ft['x_speed_std'] = df.ft.apply(lambda x : -1 if x.x_no_prev_races == 1 else x.x_speed_std, axis = 1  )
"""

print(datetime.now())


### EXPORTING THE SPEED PROFILE csv

In [35]:

print(df_base_perf.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'xr_races', 'wins', 'wins_prev', 'xr_wins', 'xr_wins_pct', 'fin_t2', 'fin_t2_prev', 'xr_fin_t2', 'xr_fin_t2_pct', 'fin_plc', 'fin_plc_prev', 'xr_fin_plc', 'xr_fin_plc_pct']


In [37]:

# 'TrainerName', 'Track' have been dummified
cols_for_drop = [ 'Box','Rug','Weight','Handicap','Margin1','Margin2','PIR','Checks','Comments','SplitMargin','Prizemoney','RaceId','TrainerId','RaceTime','Distance','RaceGrade','date'\
                , 'Place','StartPrice'\
                , 'DogName', 'TrainerName', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'position','RunTime'\
                ,'ft_sp', 'x_num_dogs'\
                , 'wins' ,'wins_prev'
                , 'fin_t2' ,'fin_t2_prev' ,'fin_plc' ,'fin_plc_prev']

## SHOULD BE THE LAST STEP OR a Separate Code
#df_base_perf1 = df_base_perf.copy()
exp_ds = df_base_perf.drop(columns = cols_for_drop)

# removed the Categorical columns from the original datast, adds the dummy variables, creates a new dataset
print(exp_ds.columns)


Index(['@id_DOG', '@id_RACE', 'xr_races', 'xr_wins', 'xr_wins_pct', 'xr_fin_t2', 'xr_fin_t2_pct', 'xr_fin_plc', 'xr_fin_plc_pct'], dtype='object')


In [38]:

exp_ds.head(11)


,@id_DOG,@id_RACE,xr_races,xr_wins,xr_wins_pct,xr_fin_t2,xr_fin_t2_pct,xr_fin_plc,xr_fin_plc_pct
3729,108390981,626271182,0,0.0,0.000000,0.0,0.000000,0.0,0.000000
9211,108390981,629797242,1,0.0,0.000000,0.0,0.000000,0.0,0.000000
14917,108390981,629797513,2,0.0,0.000000,0.0,0.000000,0.0,0.000000
37110,108390981,638810808,3,1.0,0.333333,1.0,0.333333,1.0,0.333333
48421,108390981,643554706,4,1.0,0.250000,1.0,0.250000,1.0,0.250000
2731,108391383,626218628,0,0.0,0.000000,0.0,0.000000,0.0,0.000000
5397,108391383,629829667,1,0.0,0.000000,0.0,0.000000,0.0,0.000000
10300,108391383,629829905,2,0.0,0.000000,0.0,0.000000,0.0,0.000000
13470,108391383,629797520,3,0.0,0.000000,0.0,0.000000,0.0,0.000000
46851,108391383,643552772,4,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [39]:


todaydt = datetime.now().date()
exp_fname = f'dataprep_performance_{todaydt}.csv'


In [40]:

print(exp_ds.columns.values.tolist())


['@id_DOG', '@id_RACE', 'xr_races', 'xr_wins', 'xr_wins_pct', 'xr_fin_t2', 'xr_fin_t2_pct', 'xr_fin_plc', 'xr_fin_plc_pct']


In [41]:

exp_ds.head()


,@id_DOG,@id_RACE,xr_races,xr_wins,xr_wins_pct,xr_fin_t2,xr_fin_t2_pct,xr_fin_plc,xr_fin_plc_pct
3729,108390981,626271182,0,0.0,0.000000,0.0,0.000000,0.0,0.000000
9211,108390981,629797242,1,0.0,0.000000,0.0,0.000000,0.0,0.000000
14917,108390981,629797513,2,0.0,0.000000,0.0,0.000000,0.0,0.000000
37110,108390981,638810808,3,1.0,0.333333,1.0,0.333333,1.0,0.333333
48421,108390981,643554706,4,1.0,0.250000,1.0,0.250000,1.0,0.250000


In [42]:

exp_ds.describe()


,@id_DOG,@id_RACE,xr_races,xr_wins,xr_wins_pct,xr_fin_t2,xr_fin_t2_pct,xr_fin_plc,xr_fin_plc_pct
count,5.367820e+05,5.367820e+05,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000
mean,4.526384e+08,7.292602e+08,17.441984,2.639358,0.150814,5.287484,0.293287,6.638280,0.364847
std,1.120535e+08,5.833205e+07,15.125087,3.092784,0.164388,5.404095,0.219539,6.516819,0.237093
min,1.083910e+08,6.238619e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.836043e+08,6.801957e+08,5.000000,0.000000,0.000000,1.000000,0.142857,2.000000,0.208333
50%,4.517679e+08,7.290297e+08,14.000000,2.000000,0.117647,4.000000,0.275862,5.000000,0.357143
75%,5.274578e+08,7.792436e+08,26.000000,4.000000,0.222222,8.000000,0.416667,10.000000,0.500000
max,2.145360e+09,8.292517e+08,117.000000,40.000000,1.000000,54.000000,1.000000,57.000000,1.000000


In [43]:

print(pd.DataFrame(exp_ds.isna().sum()).transpose())


   @id_DOG  @id_RACE  xr_races  xr_wins  xr_wins_pct  xr_fin_t2  xr_fin_t2_pct  xr_fin_plc  xr_fin_plc_pct
0        0         0         0        0            0          0              0           0               0


In [ ]:

exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)


In [ ]:

print('THE END')
